---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

Mise en place :
- mettre une image de travail : 'sample.jpg'
- choisir un environement de travail virtuel avec cv2
- raccourci utile de VSC : CTRL+ALT+P Pyhton (choix du bon python) - CTRL+ALT+P Clear All Output (Mets toutes les cellules à zéro)

Bonne ballade au pays des filtres !


# Initiation aux filtres avec OpenCV — v2 (NBJ)

Objectif : découvrir les filtres et opérations de base en **traitement d'image** avec **OpenCV** (convolution, lissage,
détection de contours, rehaussement, contrastes, seuillage, effets créatifs).

> Conseils :  
> - Exécute les cellules dans l'ordre.  
> - Remplace le `path_img` par une image de ton choix.  
> - Appuie sur **q** pour fermer les fenêtres `cv2.imshow`.


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# -------- Utilitaires --------

def imshow_cv(title, img):
    """Affiche une image avec cv2.imshow et gère la fermeture par 'q'."""
    cv2.imshow(title, img)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

def imshow_plt(img_bgr, title="Image (RGB via matplotlib)"):
    """Affiche une image BGR (OpenCV) en RGB via matplotlib (utile si imshow ne marche pas)."""
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def read_image(path_img):
    img = cv2.imread(str(path_img))
    if img is None:
        raise FileNotFoundError(f"Image introuvable : {path_img}\nPlace une image dans ce chemin, ou modifie path_img.")
    return img

def stack_h(*imgs):
    """Empile horizontalement des images de mêmes dimensions."""
    return np.hstack(imgs)

def stack_v(*imgs):
    """Empile verticalement des images de mêmes dimensions."""
    return np.vstack(imgs)



## 1) Charger une image

Modifie `path_img` si nécessaire. Les deux fonctions d'affichage sont fournies :  
- `imshow_cv` (fenêtre OpenCV, fermer avec **q**),  
- `imshow_plt` (affichage inline Notebook).


In [ ]:

# --- Chemin de l'image ---
# Exemple: path_img = 'data/lena.png'  (à adapter)
path_img = 'image/sample.jpg'   # <- remplace par ton image

# Charger
img = read_image(path_img)
print(img.shape, img.dtype)

# Afficher en RGB via matplotlib (pratique dans les notebooks)
imshow_plt(img, title="Image originale (RGB)")



## 2) Convolution : noyaux 3×3

On applique `cv2.filter2D` avec différents noyaux (identité, flou boîte, sharpen, Prewitt).


In [ ]:

kern_identity = np.array([[0,0,0],
                          [0,1,0],
                          [0,0,0]], dtype=np.float32)

kern_box = (1/9.0) * np.ones((3,3), dtype=np.float32)

kern_sharpen = np.array([[0,-1,0],
                         [-1,5,-1],
                         [0,-1,0]], dtype=np.float32)

# Prewitt
kern_prewitt_x = np.array([[-1,0,1],
                           [-1,0,1],
                           [-1,0,1]], dtype=np.float32)
kern_prewitt_y = np.array([[-1,-1,-1],
                           [ 0, 0, 0],
                           [ 1, 1, 1]], dtype=np.float32)

out_identity = cv2.filter2D(img, -1, kern_identity)
out_box      = cv2.filter2D(img, -1, kern_box)
out_sharp    = cv2.filter2D(img, -1, kern_sharpen)
out_pwx      = cv2.filter2D(img, -1, kern_prewitt_x)
out_pwy      = cv2.filter2D(img, -1, kern_prewitt_y)

# Aperçu matplotlib (évite d'ouvrir 5 fenêtres)
row1 = stack_h(out_identity, out_box, out_sharp)
row2 = stack_h(out_pwx, out_pwy, img)
grid = stack_v(row1, row2)
imshow_plt(grid, title="Convolution 3x3: Identité | Box | Sharpen / Prewitt X | Prewitt Y | Originale")



## 3) Lissages (anti-bruit)

Comparer : **Box**, **Gaussian**, **Median**, **Bilateral** (préserve mieux les bords).


In [ ]:

box = cv2.blur(img, (7,7))
gauss = cv2.GaussianBlur(img, (7,7), 1.5)
median = cv2.medianBlur(img, 7)
bilat = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

row = stack_h(box, gauss, median, bilat)
imshow_plt(row, title="Box | Gaussian | Median | Bilateral")



## 4) Dérivées (Sobel, Laplacien) et magnitude du gradient


In [ ]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sx = cv2.Sobel(gray, cv2.CV_16S, 1, 0, ksize=3)
sy = cv2.Sobel(gray, cv2.CV_16S, 0, 1, ksize=3)
sobel_x = cv2.convertScaleAbs(sx)
sobel_y = cv2.convertScaleAbs(sy)

lap = cv2.Laplacian(gray, cv2.CV_16S, ksize=3)
lap = cv2.convertScaleAbs(lap)

# Magnitude
mag = np.sqrt(sx.astype(np.float32)**2 + sy.astype(np.float32)**2)
mag = np.clip((mag / mag.max())*255, 0, 255).astype(np.uint8)

trip = stack_h(cv2.cvtColor(sobel_x, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(sobel_y, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(lap, cv2.COLOR_GRAY2BGR))
imshow_plt(trip, title="Sobel X | Sobel Y | Laplacien")
imshow_plt(cv2.cvtColor(mag, cv2.COLOR_GRAY2BGR), title="Magnitude du gradient")



## 5) Contours Canny


In [ ]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=100, threshold2=200)
imshow_plt(cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), title="Canny (100/200)")



## 6) Netteté : *Unsharp mask*

Plus souple qu'un simple noyau sharpen 3×3.


In [ ]:

sigma = 2.0
alpha = 1.5  # gain
blur = cv2.GaussianBlur(img, (0,0), sigmaX=sigma)
unsharp = cv2.addWeighted(img, alpha, blur, -(alpha-1.0), 0)
imshow_plt(unsharp, title=f"Unsharp mask (sigma={sigma}, alpha={alpha})")



## 7) Effet *Emboss* (relief)

Noyau 3×3 classique, avec recentrage.


In [ ]:

kern_emboss = np.array([[-2,-1,0],
                        [-1, 1,1],
                        [ 0, 1,2]], dtype=np.float32)
emb = cv2.filter2D(img, -1, kern_emboss)
emb = cv2.add(emb, 128)  # recentrer
imshow_plt(emb, title="Emboss")



## 8) Contraste local (CLAHE)

Sur le canal **L** dans l'espace **LAB** pour éviter les dérives de couleur.


In [ ]:

lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
L, A, B = cv2.split(lab)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
L2 = clahe.apply(L)
lab2 = cv2.merge([L2, A, B])
clahe_bgr = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

row = stack_h(img, clahe_bgr)
imshow_plt(row, title="Originale | CLAHE (LAB)")



## 9) Seuillage : fixe, Otsu, adaptatif


In [ ]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# binaire fixe
_, th = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

# Otsu
_, th_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Adaptatif
th_adp_mean = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                    cv2.THRESH_BINARY, blockSize=21, C=5)
th_adp_gauss = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY, blockSize=21, C=5)

row1 = stack_h(cv2.cvtColor(th, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(th_otsu, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(th_adp_mean, cv2.COLOR_GRAY2BGR))
row2 = stack_h(cv2.cvtColor(th_adp_gauss, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR),
               img)
grid = stack_v(row1, row2)
imshow_plt(grid, title="Fixe | Otsu | Adaptatif Mean / Adaptatif Gauss | Gray | Originale")



## 10) Posterization (quantification des intensités)

Réduction du nombre de niveaux (ex. 4).


In [ ]:

levels = 4
step = 256 // levels
poster = (img // step) * step
row = stack_h(img, poster)
imshow_plt(row, title=f"Originale | Posterization ({levels} niveaux)")



## 11) Négatif et Sépia


In [ ]:

neg = 255 - img

sepia_kernel = np.array([[0.272, 0.534, 0.131],
                         [0.349, 0.686, 0.168],
                         [0.393, 0.769, 0.189]], dtype=np.float32)
sepia = cv2.transform(img, sepia_kernel)
sepia = np.clip(sepia, 0, 255).astype(np.uint8)

row = stack_h(img, neg, sepia)
imshow_plt(row, title="Originale | Négatif | Sépia")



## 12) Dominante par canal (idée : *argmax* par pixel)

Pour chaque pixel, on garde uniquement le canal le plus fort (B, G ou R).


In [ ]:

b, g, r = cv2.split(img)
stack = np.stack([b, g, r], axis=-1)  # H×W×3
argmax = np.argmax(stack, axis=-1)    # 0/1/2

dom = np.zeros_like(img)
dom[argmax==0] = [255, 0, 0]   # Bleu dominant -> Bleu pur
dom[argmax==1] = [0, 255, 0]   # Vert dominant -> Vert pur
dom[argmax==2] = [0, 0, 255]   # Rouge dominant -> Rouge pur

row = stack_h(img, dom)
imshow_plt(row, title="Originale | Dominante de canal (B/G/R)")



## 13) (Bonus) Morphologie : érosion / dilatation

Utile pour nettoyer des masques binaires (après seuillage).


In [ ]:

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((3,3), np.uint8)

er = cv2.erode(th, kernel, iterations=1)
di = cv2.dilate(th, kernel, iterations=1)
op = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
cl = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)

row1 = stack_h(cv2.cvtColor(th, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(er, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(di, cv2.COLOR_GRAY2BGR))
row2 = stack_h(cv2.cvtColor(op, cv2.COLOR_GRAY2BGR),
               cv2.cvtColor(cl, cv2.COLOR_GRAY2BGR),
               img)
grid = stack_v(row1, row2)
imshow_plt(grid, title="Binaire | Érosion | Dilatation / Ouverture | Fermeture | Originale")



## 14) Exercices (à faire faire aux élèves)

1. **Comparer** Median vs Gaussian sur une image avec bruit "sel & poivre".  
2. **Composer** un pipeline : Bilateral → Canny → Dilatation (afin d'épaissir les contours).  
3. **Explorer** l'effet d'`alpha` et `sigma` sur l'Unsharp mask pour éviter la sur-accentuation.  
4. **Créer** un filtre emboss personnalisé (noyau 3×3) et commenter l'effet selon l'orientation.  
5. **Tester** CLAHE : varier `clipLimit` et `tileGridSize` et expliquer le rôle de chacun.



---

**Crédits & remarques**  
- Notebook pédagogique d'initiation (OpenCV).  
- Pour des démonstrations en direct, tu peux remplacer certains `imshow_plt` par `imshow_cv` si tu préfères des fenêtres natives.  
- Fermer les fenêtres `cv2.imshow` avec **q**.

Bon travail !


---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---